In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import json
from sklearn.preprocessing import LabelEncoder

# Define intents for the chatbot
intents = {
    "intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "Hello", "Hey", "How are you?", "What's up?"],
         "responses": ["Hello!", "Hi there!", "Greetings!", "How can I assist you?"]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "I am leaving"],
         "responses": ["Goodbye!", "See you later!", "Take care!"]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful"],
         "responses": ["You're welcome!", "Glad I could help!", "No problem!"]
        },
        {"tag": "about",
         "patterns": ["What can you do?", "Tell me about yourself", "What are you?"],
         "responses": ["I am a simple AI chatbot!", "I'm here to chat with you and answer basic questions."]
        }
    ]
}

# Prepare data for training
words = []
classes = []
documents = []
ignore_words = ["?", "!"]

# Tokenize and create word and class lists
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = pattern.lower().split()  # Split each pattern by spaces
        words.extend(word_list)  # Add tokenized words to the word list
        documents.append((word_list, intent['tag']))  # Add to document
        if intent['tag'] not in classes:
            classes.append(intent['tag'])  # Add new intent tag if not already in classes

# Lemmatize words (but here we will just keep it simple for this example)
words = sorted(set(words))  # Remove duplicates and sort

# Sort the classes
classes = sorted(set(classes))

# Prepare the training data (bag of words)
training = []
output_empty = [0] * len(classes)  # Empty list for one-hot encoding of classes

for doc in documents:
    bag = []
    word_patterns = doc[0]
    for word in words:
        bag.append(1 if word in word_patterns else 0)  # Create the bag of words representation

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1  # One-hot encoding for the intent tag
    training.append([bag, output_row])  # Add the data to the training set

# Shuffle the training data and convert to numpy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Build the neural network model
model = keras.Sequential([
    layers.Dense(128, input_shape=(len(train_x[0]),), activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(len(train_y[0]), activation="softmax")
])

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
history = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Function to process input text and clean it
def clean_up_sentence(sentence):
    sentence_words = sentence.lower().split()  # Split sentence by spaces
    return sentence_words

# Function to create a bag of words from the sentence
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)  # Clean up the sentence
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1  # Mark the word as present in the bag
    return np.array(bag)

# Function to predict the class of an input sentence
def predict_class(sentence):
    bow_vector = bow(sentence, words)  # Convert the sentence to a bag of words
    res = model.predict(np.array([bow_vector]))[0]  # Get the prediction probabilities
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)  # Sort the results
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]  # Prepare results
    return return_list

# Function to get the chatbot's response based on the predicted intent
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']  # Get the predicted tag
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])  # Return a random response from the intent

# Start chatting with the bot
print("Chatbot is ready to chat! (Type 'quit' to stop)")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    intents_list = predict_class(message)  # Predict the intent of the message
    response = get_response(intents_list, intents)  # Pass intents_json properly as the second argument
    print("Chatbot:", response)


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1833 - loss: 1.4363   
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2833 - loss: 1.4315 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2917 - loss: 1.3434 
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1583 - loss: 1.3747     
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3417 - loss: 1.3342 
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3750 - loss: 1.4140 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2667 - loss: 1.2813 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3750 - loss: 1.3355 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6000 - loss: 1.1927 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4083 - loss: 1.2525 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4750 - loss: 1.1833 
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3917